In [24]:
# Imports
import geemap
import numpy as np
import pandas as pd
import ee

In [25]:
import ee
import geemap
import ipywidgets as widgets

ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [26]:


admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
thane = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Thane'))
Map = geemap.Map()


s2 = ee.ImageCollection("COPERNICUS/S2")

rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

# Write a function for Cloud masking
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).select("B.*") \
      .copyProperties(image, ["system:time_start"])

filtered = s2 \
  .filter(ee.Filter.date('2019-01-01', '2019-12-31')) \
  .filter(ee.Filter.bounds(thane)) \
  .map(maskS2clouds)

timelapseimage = s2 \
  .filter(ee.Filter.bounds(thane)) \
  .map(maskS2clouds)

filtered2 = s2 \
  .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
  .filter(ee.Filter.bounds(thane)) \
  .map(maskS2clouds)

image2019 = filtered.median().clip(thane)
image2020 = filtered2.median().clip(thane)

# Display the input composite.

Map.addLayer(image2019, rgbVis, '2019')
Map.addLayer(image2020, rgbVis, '2020')

ndvi2019 = image2019.normalizedDifference(['B8', 'B4']).rename(['ndvi'])
ndvi2020 = image2020.normalizedDifference(['B8', 'B4']).rename(['ndvi'])
ndviVis = {'min':0, 'max':1, 'palette': ['white', 'green']}

Map.addLayer(ndvi2019, ndviVis, 'ndvi')
Map.addLayer(ndvi2020, ndviVis, 'ndvi')


# 3. Classify NDVI into 5 classes
ndvi2019classify = ee.Image(1) \
          .where(ndvi2019.gt(0.0).And(ndvi2019.lte(0.2)), 2) \
          .where(ndvi2019.gt(0.2).And(ndvi2019.lte(0.4)), 3) \
          .where(ndvi2019.gt(0.4).And(ndvi2019.lte(0.6)), 4) \
          .where(ndvi2019.gt(0.6), 5).clip(thane)

ndvi2020classify = ee.Image(1) \
          .where(ndvi2020.gt(0.0).And(ndvi2020.lte(0.2)), 2) \
          .where(ndvi2020.gt(0.2).And(ndvi2020.lte(0.4)), 3) \
          .where(ndvi2020.gt(0.4).And(ndvi2020.lte(0.6)), 4) \
          .where(ndvi2020.gt(0.6), 5).clip(thane)

# Add map layers
Map.addLayer(ndvi2019classify, {'min': 1, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Classified NDVI 2019',True)
Map.addLayer(ndvi2020classify, {'min': 1, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Classified NDVI 2020',True)


#spectral changes
def magnitude(image):
  return image.pow(2).reduce(ee.Reducer.sum()).sqrt()

distance = ndvi2020.subtract(ndvi2019)


Map.addLayer(distance, {'min': 0, 'max': 0.1, 'palette': ['white', 'green']}, 'spectral distance')

threshold=0.3

changes = distance.lte(threshold).Or(distance.gte(threshold)).selfMask()
# Calculating area above the threshold value
pixel_area=changes.multiply(ee.Image.pixelArea())
area=pixel_area.reduceRegion(reducer=ee.Reducer.sum(), geometry=thane, scale=30)

print(area.get('NDVI'))

ee.ComputedObject({
  "functionInvocationValue": {
    "functionName": "Dictionary.get",
    "arguments": {
      "dictionary": {
        "functionInvocationValue": {
          "functionName": "Image.reduceRegion",
          "arguments": {
            "geometry": {
              "functionInvocationValue": {
                "functionName": "Collection.geometry",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "Collection.loadTable",
                            "arguments": {
                              "tableId": {
                                "constantValue": "FAO/GAUL_SIMPLIFIED_500m/2015/level2"
                              }
                            }
                          }

In [27]:
# With widgets 
Map = geemap.Map()

style = {"description_width":"initial"}

start_year = widgets.IntSlider(description='Start Year:', value=1984, min=1984, max=2020, style=style)
end_year = widgets.IntSlider(description='End Year:', value=2020, min=1984, max=2020, style=style)
start_month = widgets.IntSlider(description='Start Month:', value=5, min=1, max=12, style=style)
end_month = widgets.IntSlider(description='End Month:', value=10, min=1, max=12, style=style)



hbox3 = widgets.HBox([start_year, start_month])
hbox4 = widgets.HBox([end_year, end_month])

In [28]:
#Timelapse widgets

create_gif = widgets.Button(
    description='Create timelapse',
    button_style='primary',
    tooltip='Click to create timelapse',
    style=style
)

download_gif = widgets.Button(
    description='Download GIF',
    button_style='primary',
    tooltip='Click to download timelapse',
    disabled=False,
    style=style
)

output = widgets.Output()

hbox5 = widgets.HBox([create_gif])

In [29]:
#On submitting the data from widgets

def submit_clicked(b):
    
    with output:
        output.clear_output()
        if start_year.value > end_year.value:
            print('The end year must be great than the start year.')
            return
        if start_month.value > end_month.value:
            print('The end month must be great than the start month.')
            return        
        if start_year.value == end_year.value:
            add_progress_bar = False
        else:
            add_progress_bar = True
            
        start_date = str(start_month.value).zfill(2) + '-01'
        end_date = str(end_month.value).zfill(2) + '-30'
        
        print('Computing...')
        label="Timelapse for NDVI"
        roi= ee.Geometry.Polygon(
        [[[73.04124316585163, 18.97918033662206],
          [73.94761523616413, 19.33721584963624],
          [73.31590136897663, 20.308722406734972],
          [72.53037890803913, 20.154094798779248],
          [72.64573535335163, 19.311297428069476]]]);
        Map.add_landsat_ts_gif(label= label,roi=roi, bands=['Blue', 'NIR', 'Red'], nd_bands=['NIR', 'Red'], nd_palette=['blue', 'white', 'green'], nd_threshold=0, start_year=1985, end_year=2020,start_date="01-01", end_date="12-31",add_progress_bar= True, frames_per_second=3)



create_gif.on_click(submit_clicked)

In [30]:
label="Timelapse for NDVI"
roi= ee.Geometry.Polygon(
        [[[73.04124316585163, 18.97918033662206],
          [73.94761523616413, 19.33721584963624],
          [73.31590136897663, 20.308722406734972],
          [72.53037890803913, 20.154094798779248],
          [72.64573535335163, 19.311297428069476]]]);
Map.add_landsat_ts_gif(label= label,roi=roi, bands=['Blue', 'NIR', 'Red'], nd_bands=['NIR', 'Red'], nd_palette=['blue', 'white', 'green'], nd_threshold=0, start_year=1985, end_year=2020,start_date="01-01", end_date="12-31",add_progress_bar= True, frames_per_second=3)


Generating URL...
Please wait ...
The GIF image has been saved to: /var/folders/83/rcfkd72n2k3cmzcj288kt_h80000gn/T/2585869a-f126-4b07-9ebe-96d9321facbf.gif
Generating URL...
Please wait ...
The GIF image has been saved to: /var/folders/83/rcfkd72n2k3cmzcj288kt_h80000gn/T/2585869a-f126-4b07-9ebe-96d9321facbf_nd.gif
Adding GIF to the map ...
The timelapse has been added to the map.


In [31]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [32]:
output

Output()

In [33]:
hbox3

In [34]:
hbox4

In [35]:
hbox5

In [36]:
javascript_code = """
   var bangalore = ee.FeatureCollection('users/ujavalgandhi/public/bangalore_boundary');
    var change = ee.FeatureCollection('users/ujavalgandhi/e2e/bangalore_change_gcps');
    var nochange = ee.FeatureCollection('users/ujavalgandhi/e2e/bangalore_nochange_gcps')
    var s2 = ee.ImageCollection("COPERNICUS/S2")

    var rgbVis = {
    min: 0.0,
    max: 3000,
    bands: ['B4', 'B3', 'B2'],
    };

    // Write a function for Cloud masking
    function maskS2clouds(image) {
    var qa = image.select('QA60')
    var cloudBitMask = 1 << 10;
    var cirrusBitMask = 1 << 11;
    var mask = qa.bitwiseAnd(cloudBitMask).eq(0).and(
                qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)//.divide(10000)
        .select("B.*")
        .copyProperties(image, ["system:time_start"])
    } 

    var filtered = s2
    .filter(ee.Filter.date('2019-01-01', '2019-02-01'))
    .filter(ee.Filter.bounds(bangalore))
    .map(maskS2clouds)

    
    var image2019 = filtered.median().clip(bangalore)
    // Display the input composite.
    Map.addLayer(image2019, rgbVis, '2019');

    var filtered = s2
    .filter(ee.Filter.date('2020-01-01', '2020-02-01'))
    .filter(ee.Filter.bounds(bangalore))
    .map(maskS2clouds)

    var image2020 = filtered.median().clip(bangalore)

    Map.addLayer(image2020, rgbVis, '2020');

    var stackedImage = image2019.addBands(image2020)

    // Overlay the point on the image to get training data.
    var training = stackedImage.sampleRegions({
    collection: change.merge(nochange), 
    properties: ['class'], 
    scale: 10
    });

    // Train a classifier.
    var classifier = ee.Classifier.smileRandomForest(50).train({
    features: training,  
    classProperty: 'class', 
    inputProperties: stackedImage.bandNames()
    });

    // Classify the image.
    var classified = stackedImage.classify(classifier);
    Map.addLayer(classified, {min: 0, max: 1, palette: ['white', 'red']}, 'change'); 
"""
    

In [37]:
geemap.js_snippet_to_py(
    javascript_code, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True
)

In [ ]:
import ee
import geemap

Map = geemap.Map()

   bangalore = ee.FeatureCollection('users/ujavalgandhi/public/bangalore_boundary')
    change = ee.FeatureCollection('users/ujavalgandhi/e2e/bangalore_change_gcps')
    nochange = ee.FeatureCollection('users/ujavalgandhi/e2e/bangalore_nochange_gcps')
    s2 = ee.ImageCollection("COPERNICUS/S2")

    rgbVis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
    }

    # Write a function for Cloud masking
    def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
                qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)#.divide(10000) \
        .select("B.*") \
        .copyProperties(image, ["system:time_start"])

    filtered = s2 \
    .filter(ee.Filter.date('2019-01-01', '2019-02-01')) \
    .filter(ee.Filter.bounds(bangalore)) \
    .map(maskS2clouds)

    image2019 = filtered.median().clip(bangalore)
    # Display the input composite.
    Map.addLayer(image2019, rgbVis, '2019')

    filtered = s2 \
    .filter(ee.Filter.date('2020-01-01', '2020-02-01')) \
    .filter(ee.Filter.bounds(bangalore)) \
    .map(maskS2clouds)

    image2020 = filtered.median().clip(bangalore)

    Map.addLayer(image2020, rgbVis, '2020')

    stackedImage = image2019.addBands(image2020)

    # Overlay the point on the image to get training data.
    training = stackedImage.sampleRegions({
    'collection': change.merge(nochange),
    'properties': ['class'],
    'scale': 10
    })

    # Train a classifier.
    classifier = ee.Classifier.smileRandomForest(50).train({
    'features': training,
    'classProperty': 'class',
    'inputProperties': stackedImage.bandNames()
    })

    # Classify the image.
    classified = stackedImage.classify(classifier)
    Map.addLayer(classified, {'min': 0, 'max': 1, 'palette': ['white', 'red']}, 'change')
Map


In [38]:
import ee
import geemap

Map = geemap.Map()

    bangalore = ee.FeatureCollection('users/ujavalgandhi/public/bangalore_boundary')
    change = ee.FeatureCollection('users/ujavalgandhi/e2e/bangalore_change_gcps')
    nochange = ee.FeatureCollection('users/ujavalgandhi/e2e/bangalore_nochange_gcps')
    s2 = ee.ImageCollection("COPERNICUS/S2")

    rgbVis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
    }

    # Write a function for Cloud masking
    def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
                qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)#.divide(10000) \
        .select("B.*") \
        .copyProperties(image, ["system:time_start"])

    filtered = s2 \
    .filter(ee.Filter.date('2019-01-01', '2019-02-01')) \
    .filter(ee.Filter.bounds(bangalore)) \
    .map(maskS2clouds)

    image2019 = filtered.median().clip(bangalore)
    # Display the input composite.
    Map.addLayer(image2019, rgbVis, '2019')

    filtered = s2 \
    .filter(ee.Filter.date('2020-01-01', '2020-02-01')) \
    .filter(ee.Filter.bounds(bangalore)) \
    .map(maskS2clouds)

    image2020 = filtered.median().clip(bangalore)

    Map.addLayer(image2020, rgbVis, '2020')

    stackedImage = image2019.addBands(image2020)

    # Overlay the point on the image to get training data.
    training = stackedImage.sampleRegions({
    'collection': change.merge(nochange),
    'properties': ['class'],
    'scale': 10
    })

    # Train a classifier.
    classifier = ee.Classifier.smileRandomForest(50).train({
    'features': training,
    'classProperty': 'class',
    'inputProperties': stackedImage.bandNames()
    })

    # Classify the image.
    classified = stackedImage.classify(classifier)
    Map.addLayer(classified, {'min': 0, 'max': 1, 'palette': ['white', 'red']}, 'change')
Map


IndentationError: unexpected indent (3775963078.py, line 6)

In [ ]:

features_js="""var veg = /* color: #d63000 */ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([72.98510261651016, 19.129642846679292]),
            {
              "lc": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([73.04853149529434, 19.195381276362475]),
            {
              "lc": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([73.0384034740541, 19.19335479515654]),
            {
              "lc": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([73.05224121566928, 19.19815633475566]),
            {
              "lc": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([73.03138435836948, 19.18786172524441]),
            {
              "lc": 0,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([73.03722084518589, 19.170837730468406]),
            {
              "lc": 0,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([73.30962599273838, 19.140169837198044]),
            {
              "lc": 0,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([73.31316650863803, 19.138608926142737]),
            {
              "lc": 0,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([73.31874550338901, 19.136034404237634]),
            {
              "lc": 0,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([73.31591309066928, 19.136642562639665]),
            {
              "lc": 0,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([73.33215654846347, 19.151784984417244]),
            {
              "lc": 0,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([73.3375638818375, 19.152028224833522]),
            {
              "lc": 0,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([73.32649172302402, 19.154055214355385]),
            {
              "lc": 0,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([73.34657610412754, 19.157379423250784]),
            {
              "lc": 0,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([73.31808031555332, 19.172540209112217]),
            {
              "lc": 0,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([73.02955923751342, 19.425622552126892]),
            {
              "lc": 0,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([72.91969595626342, 19.41914684768232]),
            {
              "lc": 0,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([73.08037100509155, 19.914433291337808]),
            {
              "lc": 0,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([72.93342886641967, 19.782679069694858]),
            {
              "lc": 0,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([73.05625258162964, 19.962764130201126]),
            {
              "lc": 0,
              "system:index": "19"
            })]),
    land = /* color: #98ff00 */ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([73.25185723872947, 19.97964297367706]),
            {
              "lc": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([73.28155465694236, 19.983434177879946]),
            {
              "lc": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([73.10426681063868, 20.04528545371028]),
            {
              "lc": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([73.09290497325159, 20.063325656397218]),
            {
              "lc": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([73.08607070468165, 20.084336349779445]),
            {
              "lc": 1,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([73.08592050097681, 20.083389165704343]),
            {
              "lc": 1,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([73.08171479724146, 20.0848401687634]),
            {
              "lc": 1,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([73.07961194537378, 20.06863653877081]),
            {
              "lc": 1,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([72.82536989521253, 20.003903117789914]),
            {
              "lc": 1,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([72.82189911674726, 20.004573539974185]),
            {
              "lc": 1,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([72.7994847895548, 20.018329002847953]),
            {
              "lc": 1,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([72.79106063023907, 20.01032506318204]),
            {
              "lc": 1,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([72.78036737681862, 19.79593050301516]),
            {
              "lc": 1,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([72.77901017905708, 19.794885686376226]),
            {
              "lc": 1,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([72.78031909705635, 19.797040928640612]),
            {
              "lc": 1,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([73.0496018181072, 19.572605750710924]),
            {
              "lc": 1,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([73.07854821779593, 19.574243384119992]),
            {
              "lc": 1,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([73.07825708454082, 19.568189216540222]),
            {
              "lc": 1,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([73.07282829349468, 19.561800090327154]),
            {
              "lc": 1,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([73.07427132194469, 19.571328085307325]),
            {
              "lc": 1,
              "system:index": "19"
            })]),
    water = /* color: #0b4a8b */ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([73.06412278267406, 19.20576492578159]),
            {
              "lc": 2,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([73.1453881800071, 19.26791799445695]),
            {
              "lc": 2,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([73.16942077278054, 19.250578164740105]),
            {
              "lc": 2,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([73.19928985237038, 19.261517055868023]),
            {
              "lc": 2,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15663200019753, 19.272698279216346]),
            {
              "lc": 2,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15354209541238, 19.27172602920541]),
            {
              "lc": 2,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15500121711648, 19.272698279216346]),
            {
              "lc": 2,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15328460334695, 19.27075377342604]),
            {
              "lc": 2,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15972190498269, 19.27342746293887]),
            {
              "lc": 2,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([73.18052258115847, 19.28441500094283]),
            {
              "lc": 2,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([73.1793209515198, 19.287412521065736]),
            {
              "lc": 2,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([73.19653000455935, 19.292475775005162]),
            {
              "lc": 2,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([73.17902225636686, 19.30157285450608]),
            {
              "lc": 2,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([73.17721981190886, 19.300114734488048]),
            {
              "lc": 2,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([73.1750311293527, 19.29760349732846]),
            {
              "lc": 2,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([73.17430156850065, 19.29533524999376]),
            {
              "lc": 2,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([73.22652954243864, 19.311131319402485]),
            {
              "lc": 2,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([73.26553228354324, 19.303772755849163]),
            {
              "lc": 2,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([73.22072866415847, 19.300654025253678]),
            {
              "lc": 2,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([73.01447291675183, 19.471009417777175]),
            {
              "lc": 2,
              "system:index": "19"
            })]),
    urban = /* color: #ffc82d */ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([73.16207398855612, 19.22032280462053]),
            {
              "lc": 3,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([73.1629001089327, 19.22038358945196]),
            {
              "lc": 3,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([73.16389252626821, 19.2199226372525]),
            {
              "lc": 3,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([73.16427339994833, 19.21985172141476]),
            {
              "lc": 3,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15880860795713, 19.218524037781354]),
            {
              "lc": 3,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15790738572812, 19.21884822683725]),
            {
              "lc": 3,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15780009736753, 19.220296938867108]),
            {
              "lc": 3,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([73.16169466485715, 19.2091526715288]),
            {
              "lc": 3,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15753187646604, 19.211807101922844]),
            {
              "lc": 3,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15625514497495, 19.210682520231586]),
            {
              "lc": 3,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15628733148313, 19.20997332061585]),
            {
              "lc": 3,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([73.16669430246091, 19.21243524330335]),
            {
              "lc": 3,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([73.18856616446165, 19.203539410585286]),
            {
              "lc": 3,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([73.18818260857252, 19.203577404985673]),
            {
              "lc": 3,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([73.1879814428964, 19.203691388134175]),
            {
              "lc": 3,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([73.1882335705438, 19.203577404985673]),
            {
              "lc": 3,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([73.19028814264921, 19.20394215078289]),
            {
              "lc": 3,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([73.18978925177244, 19.20502118569752]),
            {
              "lc": 3,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([73.19656987616209, 19.20432209346162]),
            {
              "lc": 3,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([73.17925353476194, 19.198790041433597]),
            {
              "lc": 3,
              "system:index": "19"
            })]);"""

In [ ]:

javascript_classification = """
var admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2");
var thane = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Thane'))
var s2 = ee.ImageCollection("COPERNICUS/S2")

var rgbVis = {
  min: 0.0,
  max: 3000,
  bands: ['B4', 'B3', 'B2'],
};

// Write a function for Cloud masking
function maskS2clouds(image) {
  var qa = image.select('QA60')
  var cloudBitMask = 1 << 10;
  var cirrusBitMask = 1 << 11;
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0).and(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask)//.divide(10000)
      .select("B.*")
      .copyProperties(image, ["system:time_start"])
} 

var filtered = s2
  .filter(ee.Filter.date('2019-01-01', '2019-12-31'))
  .filter(ee.Filter.bounds(thane))
  .map(maskS2clouds).median().clip(thane)
  

var bands = ['B4','B3','B2','B1']

Map.addLayer(filtered,rgbVis,'Thane');
var merged=veg.merge(land).merge(water).merge(urban)

var sampled= filtered.sampleRegions({collection:merged,properties:['lc'],scale:10})
print(sampled)
var random_column=sampled.randomColumn()
var training_sample=random_column.filter(ee.Filter.gte('random',0.3))
var testing_sample=random_column.filter(ee.Filter.lte('random',0.3))

var classifier = ee.Classifier.smileRandomForest(50).train({features:training_sample,classProperty:'lc',inputProperties:bands})
var classified=filtered.classify(classifier)
Map.addLayer(classified,{min:0,max:3,palette:['green','yellow','blue','white']},'Classified image')"""

In [ ]:
geemap.js_snippet_to_py(
    javascript_classification, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True
)

In [46]:
import ee
import geemap

Map = geemap.Map()

admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
thane = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Thane'))
s2 = ee.ImageCollection("COPERNICUS/S2")

rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

# Write a function for Cloud masking
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).select("B.*").copyProperties(image, ["system:time_start"])

filtered = s2 \
  .filter(ee.Filter.date('2019-01-01', '2019-12-31')) .filter(ee.Filter.bounds(thane)) \
  .map(maskS2clouds).median().clip(thane)

bands = ['B4','B3','B2','B1']
Map.addLayer(filtered,rgbVis,'Thane')

merged = veg.merge(land).merge(water).merge(urban)
sampled= filtered.sampleRegions({'collection':merged, 'properties':['lc'], 'scale':10})
print(sampled)
random_column=sampled.randomColumn()
training_sample=random_column.filter(ee.Filter.gte('random',0.3))
testing_sample=random_column.filter(ee.Filter.lte('random',0.3))

classifier = ee.Classifier.smileRandomForest(50).train({'features':training_sample, 'classProperty':'lc', 'inputProperties':bands})
classified=filtered.classify(classifier)
Map.addLayer(classified,{'min':0, 'max':3, 'palette':['green','yellow','blue','white']},'Classified image')
Map


EEException: Unrecognized argument type to convert to a FeatureCollection: {'collection': <ee.featurecollection.FeatureCollection object at 0x168c8e700>, 'properties': ['lc'], 'scale': 10}

In [ ]:
geemap.js_snippet_to_py(
    , add_new_cell=True, import_ee=True, import_geemap=True, show_map=True
)

In [ ]:
import ee
import geemap

Map = geemap.Map()

veg = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([72.98510261651016, 19.129642846679292]),
            {
              "lc": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([73.04853149529434, 19.195381276362475]),
            {
              "lc": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([73.0384034740541, 19.19335479515654]),
            {
              "lc": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([73.05224121566928, 19.19815633475566]),
            {
              "lc": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([73.03138435836948, 19.18786172524441]),
            {
              "lc": 0,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([73.03722084518589, 19.170837730468406]),
            {
              "lc": 0,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([73.30962599273838, 19.140169837198044]),
            {
              "lc": 0,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([73.31316650863803, 19.138608926142737]),
            {
              "lc": 0,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([73.31874550338901, 19.136034404237634]),
            {
              "lc": 0,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([73.31591309066928, 19.136642562639665]),
            {
              "lc": 0,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([73.33215654846347, 19.151784984417244]),
            {
              "lc": 0,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([73.3375638818375, 19.152028224833522]),
            {
              "lc": 0,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([73.32649172302402, 19.154055214355385]),
            {
              "lc": 0,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([73.34657610412754, 19.157379423250784]),
            {
              "lc": 0,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([73.31808031555332, 19.172540209112217]),
            {
              "lc": 0,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([73.02955923751342, 19.425622552126892]),
            {
              "lc": 0,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([72.91969595626342, 19.41914684768232]),
            {
              "lc": 0,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([73.08037100509155, 19.914433291337808]),
            {
              "lc": 0,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([72.93342886641967, 19.782679069694858]),
            {
              "lc": 0,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([73.05625258162964, 19.962764130201126]),
            {
              "lc": 0,
              "system:index": "19"
            })]),
land = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([73.25185723872947, 19.97964297367706]),
            {
              "lc": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([73.28155465694236, 19.983434177879946]),
            {
              "lc": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([73.10426681063868, 20.04528545371028]),
            {
              "lc": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([73.09290497325159, 20.063325656397218]),
            {
              "lc": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([73.08607070468165, 20.084336349779445]),
            {
              "lc": 1,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([73.08592050097681, 20.083389165704343]),
            {
              "lc": 1,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([73.08171479724146, 20.0848401687634]),
            {
              "lc": 1,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([73.07961194537378, 20.06863653877081]),
            {
              "lc": 1,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([72.82536989521253, 20.003903117789914]),
            {
              "lc": 1,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([72.82189911674726, 20.004573539974185]),
            {
              "lc": 1,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([72.7994847895548, 20.018329002847953]),
            {
              "lc": 1,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([72.79106063023907, 20.01032506318204]),
            {
              "lc": 1,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([72.78036737681862, 19.79593050301516]),
            {
              "lc": 1,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([72.77901017905708, 19.794885686376226]),
            {
              "lc": 1,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([72.78031909705635, 19.797040928640612]),
            {
              "lc": 1,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([73.0496018181072, 19.572605750710924]),
            {
              "lc": 1,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([73.07854821779593, 19.574243384119992]),
            {
              "lc": 1,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([73.07825708454082, 19.568189216540222]),
            {
              "lc": 1,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([73.07282829349468, 19.561800090327154]),
            {
              "lc": 1,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([73.07427132194469, 19.571328085307325]),
            {
              "lc": 1,
              "system:index": "19"
            })]),
water = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([73.06412278267406, 19.20576492578159]),
            {
              "lc": 2,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([73.1453881800071, 19.26791799445695]),
            {
              "lc": 2,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([73.16942077278054, 19.250578164740105]),
            {
              "lc": 2,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([73.19928985237038, 19.261517055868023]),
            {
              "lc": 2,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15663200019753, 19.272698279216346]),
            {
              "lc": 2,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15354209541238, 19.27172602920541]),
            {
              "lc": 2,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15500121711648, 19.272698279216346]),
            {
              "lc": 2,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15328460334695, 19.27075377342604]),
            {
              "lc": 2,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15972190498269, 19.27342746293887]),
            {
              "lc": 2,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([73.18052258115847, 19.28441500094283]),
            {
              "lc": 2,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([73.1793209515198, 19.287412521065736]),
            {
              "lc": 2,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([73.19653000455935, 19.292475775005162]),
            {
              "lc": 2,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([73.17902225636686, 19.30157285450608]),
            {
              "lc": 2,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([73.17721981190886, 19.300114734488048]),
            {
              "lc": 2,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([73.1750311293527, 19.29760349732846]),
            {
              "lc": 2,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([73.17430156850065, 19.29533524999376]),
            {
              "lc": 2,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([73.22652954243864, 19.311131319402485]),
            {
              "lc": 2,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([73.26553228354324, 19.303772755849163]),
            {
              "lc": 2,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([73.22072866415847, 19.300654025253678]),
            {
              "lc": 2,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([73.01447291675183, 19.471009417777175]),
            {
              "lc": 2,
              "system:index": "19"
            })]),
urban = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([73.16207398855612, 19.22032280462053]),
            {
              "lc": 3,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([73.1629001089327, 19.22038358945196]),
            {
              "lc": 3,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([73.16389252626821, 19.2199226372525]),
            {
              "lc": 3,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([73.16427339994833, 19.21985172141476]),
            {
              "lc": 3,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15880860795713, 19.218524037781354]),
            {
              "lc": 3,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15790738572812, 19.21884822683725]),
            {
              "lc": 3,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15780009736753, 19.220296938867108]),
            {
              "lc": 3,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([73.16169466485715, 19.2091526715288]),
            {
              "lc": 3,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15753187646604, 19.211807101922844]),
            {
              "lc": 3,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15625514497495, 19.210682520231586]),
            {
              "lc": 3,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([73.15628733148313, 19.20997332061585]),
            {
              "lc": 3,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([73.16669430246091, 19.21243524330335]),
            {
              "lc": 3,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([73.18856616446165, 19.203539410585286]),
            {
              "lc": 3,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([73.18818260857252, 19.203577404985673]),
            {
              "lc": 3,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([73.1879814428964, 19.203691388134175]),
            {
              "lc": 3,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([73.1882335705438, 19.203577404985673]),
            {
              "lc": 3,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([73.19028814264921, 19.20394215078289]),
            {
              "lc": 3,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([73.18978925177244, 19.20502118569752]),
            {
              "lc": 3,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([73.19656987616209, 19.20432209346162]),
            {
              "lc": 3,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([73.17925353476194, 19.198790041433597]),
            {
              "lc": 3,
              "system:index": "19"
            })])
            

In [ ]:
geemap.js_snippet_to_py(
    "var result = op1.merge(op2)", add_new_cell=True, import_ee=True, import_geemap=True, show_map=True
)

In [ ]:
import ee
import geemap

Map = geemap.Map()

result = op1.merge(op2)
Map
